In [28]:
import torch

def bbox_overlaps(bboxes1, bboxes2, mode="iou", is_aligned=False, eps=1e-6):
    assert mode in ["iou", "iof", "giou"], f"Unsupported mode {mode}"
    # Either the boxes are empty or the length of boxes's last dimenstion is 4
    assert bboxes1.size(-1) == 4 or bboxes1.size(0) == 0
    assert bboxes2.size(-1) == 4 or bboxes2.size(0) == 0

    # Batch dim must be the same
    # Batch dim: (B1, B2, ... Bn)
    assert bboxes1.shape[:-2] == bboxes2.shape[:-2]
    batch_shape = bboxes1.shape[:-2]

    rows = bboxes1.size(-2)
    cols = bboxes2.size(-2)
    if is_aligned:
        assert rows == cols

    if rows * cols == 0:
        if is_aligned:
            return bboxes1.new(batch_shape + (rows,))
        else:
            return bboxes1.new(batch_shape + (rows, cols))

    area1 = (bboxes1[..., 2] - bboxes1[..., 0]) * (bboxes1[..., 3] - bboxes1[..., 1])
    area2 = (bboxes2[..., 2] - bboxes2[..., 0]) * (bboxes2[..., 3] - bboxes2[..., 1])

    if is_aligned:
        lt = torch.max(bboxes1[..., :2], bboxes2[..., :2])  # [B, rows, 2]
        rb = torch.min(bboxes1[..., 2:], bboxes2[..., 2:])  # [B, rows, 2]

        wh = (rb - lt).clamp(min=0)  # [B, rows, 2]
        overlap = wh[..., 0] * wh[..., 1]

        if mode in ["iou", "giou"]:
            union = area1 + area2 - overlap
        else:
            union = area1
        if mode == "giou":
            enclosed_lt = torch.min(bboxes1[..., :2], bboxes2[..., :2])
            enclosed_rb = torch.max(bboxes1[..., 2:], bboxes2[..., 2:])
    else:
        lt = torch.max(
            bboxes1[..., :, None, :2], bboxes2[..., None, :, :2]
        )  # [B, rows, cols, 2]
        rb = torch.min(
            bboxes1[..., :, None, 2:], bboxes2[..., None, :, 2:]
        )  # [B, rows, cols, 2]

        wh = (rb - lt).clamp(min=0)  # [B, rows, cols, 2]
        overlap = wh[..., 0] * wh[..., 1]

        if mode in ["iou", "giou"]:
            union = area1[..., None] + area2[..., None, :] - overlap
        else:
            union = area1[..., None]
        if mode == "giou":
            enclosed_lt = torch.min(
                bboxes1[..., :, None, :2], bboxes2[..., None, :, :2]
            )
            enclosed_rb = torch.max(
                bboxes1[..., :, None, 2:], bboxes2[..., None, :, 2:]
            )

    eps = union.new_tensor([eps])
    union = torch.max(union, eps)
    ious = overlap / union
    if mode in ["iou", "iof"]:
        return ious
    # calculate gious
    enclose_wh = (enclosed_rb - enclosed_lt).clamp(min=0)
    enclose_area = enclose_wh[..., 0] * enclose_wh[..., 1]
    enclose_area = torch.max(enclose_area, eps)
    gious = ious - (enclose_area - union) / enclose_area
    return gious

def assign(
    bboxes, num_level_bboxes, gt_bboxes, gt_bboxes_ignore=None, gt_labels=None
):
    """Assign gt to bboxes.
    The assignment is done in following steps

    1. compute iou between all bbox (bbox of all pyramid levels) and gt
    2. compute center distance between all bbox and gt
    3. on each pyramid level, for each gt, select k bbox whose center
       are closest to the gt center, so we total select k*l bbox as
       candidates for each gt
    4. get corresponding iou for the these candidates, and compute the
       mean and std, set mean + std as the iou threshold
    5. select these candidates whose iou are greater than or equal to
       the threshold as postive
    6. limit the positive sample's center in gt


    Args:
        bboxes (Tensor): Bounding boxes to be assigned, shape(n, 4).
        num_level_bboxes (List): num of bboxes in each level
        gt_bboxes (Tensor): Groundtruth boxes, shape (k, 4).
        gt_bboxes_ignore (Tensor, optional): Ground truth bboxes that are
            labelled as `ignored`, e.g., crowd boxes in COCO.
        gt_labels (Tensor, optional): Label of gt_bboxes, shape (k, ).

    Returns:
        :obj:`AssignResult`: The assign result.
    """
    topk = 9
    INF = 100000000
    bboxes = bboxes[:, :4]
    num_gt, num_bboxes = gt_bboxes.size(0), bboxes.size(0)

    # compute iou between all bbox and gt
    overlaps = bbox_overlaps(bboxes, gt_bboxes)

    # assign 0 by default
    assigned_gt_inds = overlaps.new_full((num_bboxes,), 0, dtype=torch.long)

    # 基本不执行
    if num_gt == 0 or num_bboxes == 0:
        # No ground truth or boxes, return empty assignment
        max_overlaps = overlaps.new_zeros((num_bboxes,))
        if num_gt == 0:
            # No truth, assign everything to background
            assigned_gt_inds[:] = 0
        if gt_labels is None:
            assigned_labels = None
        else:
            assigned_labels = overlaps.new_full((num_bboxes,), -1, dtype=torch.long)
        return num_gt, assigned_gt_inds, max_overlaps, assigned_labels


    # compute center distance between all bbox and gt
    gt_cx = (gt_bboxes[:, 0] + gt_bboxes[:, 2]) / 2.0
    gt_cy = (gt_bboxes[:, 1] + gt_bboxes[:, 3]) / 2.0
    gt_points = torch.stack((gt_cx, gt_cy), dim=1)

    bboxes_cx = (bboxes[:, 0] + bboxes[:, 2]) / 2.0
    bboxes_cy = (bboxes[:, 1] + bboxes[:, 3]) / 2.0
    bboxes_points = torch.stack((bboxes_cx, bboxes_cy), dim=1)

    # L2距离
    distances = (
        (bboxes_points[:, None, :] - gt_points[None, :, :]).pow(2).sum(-1).sqrt()
    )

    # Selecting candidates based on the center distance
    candidate_idxs = []
    start_idx = 0
    for level, bboxes_per_level in enumerate(num_level_bboxes):
        # on each pyramid level, for each gt,
        # select k bbox whose center are closest to the gt center
        end_idx = start_idx + bboxes_per_level
        distances_per_level = distances[start_idx:end_idx, :]
        selectable_k = min(topk, bboxes_per_level)
        # 第二个输出是坐标
        _, topk_idxs_per_level = distances_per_level.topk(
            selectable_k, dim=0, largest=False
        )
        candidate_idxs.append(topk_idxs_per_level + start_idx)
        start_idx = end_idx
    candidate_idxs = torch.cat(candidate_idxs, dim=0)

    # get corresponding iou for the these candidates, and compute the
    # mean and std, set mean + std as the iou threshold
    candidate_overlaps = overlaps[candidate_idxs, torch.arange(num_gt)]

    overlaps_mean_per_gt = candidate_overlaps.mean(0)
    overlaps_std_per_gt = candidate_overlaps.std(0)

    overlaps_thr_per_gt = overlaps_mean_per_gt + overlaps_std_per_gt
    is_pos = candidate_overlaps >= overlaps_thr_per_gt[None, :]

    # limit the positive sample's center in gt
    for gt_idx in range(num_gt):
        candidate_idxs[:, gt_idx] += gt_idx * num_bboxes

    ep_bboxes_cx = (
        bboxes_cx.view(1, -1).expand(num_gt, num_bboxes).contiguous().view(-1)
    )
    ep_bboxes_cy = (
        bboxes_cy.view(1, -1).expand(num_gt, num_bboxes).contiguous().view(-1)
    )

    # calculate the left, top, right, bottom distance between positive
    # bbox center and gt side
    candidate_idxs = candidate_idxs.view(-1)
    l_ = ep_bboxes_cx[candidate_idxs].view(-1, num_gt) - gt_bboxes[:, 0]
    t_ = ep_bboxes_cy[candidate_idxs].view(-1, num_gt) - gt_bboxes[:, 1]
    r_ = gt_bboxes[:, 2] - ep_bboxes_cx[candidate_idxs].view(-1, num_gt)
    b_ = gt_bboxes[:, 3] - ep_bboxes_cy[candidate_idxs].view(-1, num_gt)
    is_in_gts = torch.stack([l_, t_, r_, b_], dim=1).min(dim=1)[0] > 0.01
    is_pos = is_pos & is_in_gts

    # if an anchor box is assigned to multiple gts,
    # the one with the highest IoU will be selected.
    overlaps_inf = torch.full_like(overlaps, -INF).t().contiguous().view(-1)
    index = candidate_idxs.view(-1)[is_pos.view(-1)]
    overlaps_inf[index] = overlaps.t().contiguous().view(-1)[index]
    overlaps_inf = overlaps_inf.view(num_gt, -1).t()

    # 挑选出这个点上，最契合的gt
    # 每个点上对应与每个gtIou最大的值, 对应GT的坐标
    max_overlaps, argmax_overlaps = overlaps_inf.max(dim=1)

    # ？特征图上是正样本的点对应的第几个GT
    #  assigned_gt_inds = overlaps.new_full((num_bboxes,), 0, dtype=torch.long)
    # 负样本上assigned_gt_inds[idx] == 0
    # 正样本上assigned_gt_inds[idx] == 第几个gt
    assigned_gt_inds[max_overlaps != -INF] = (
        argmax_overlaps[max_overlaps != -INF] + 1
    )

    if gt_labels is not None:
        # 为每个点对应label, -1表示负样本, 不对应label
        assigned_labels = assigned_gt_inds.new_full((num_bboxes,), -1)
        # assigned_gt_inds经过+1之后1代表第一个GT, 2代表第二个GT, 以此类推, 0表示负样本,不分配GT
        pos_inds = torch.nonzero(assigned_gt_inds > 0, as_tuple=False).squeeze()
        # 在0-2100中,有被分配GT的正样本下标
        if pos_inds.numel() > 0:
            # 如果筛出len>0
            # assigned_labels = assigned_gt_inds.new_full((num_bboxes,), -1)
            # 为下表为正样本pos_inds, 从gt_labels中赋值给真正的labels标识
            # 其余没有被赋值的,-1表示负样本, 既没有分配GT, 也不会给labels类别
            assigned_labels[pos_inds] = gt_labels[assigned_gt_inds[pos_inds] - 1]
    else:
        assigned_labels = None
    # num_gt, assigned_gt_inds, max_overlaps, labels=assigned_labels
    # def __init__(self, num_gts, gt_inds, max_overlaps, labels=None):
    #         self.num_gts = num_gts gtbox数量
    #         self.gt_inds = gt_inds assigned_gt_inds 表示一个点分配第几个GT, 0表示没有分配GT
    #         self.max_overlaps = max_overlaps 每个点对应若干个GT iou最大是多少 size=(num_bboxes,)
    #         self.labels = labels assigned_labels 为每个点对应的GT分配正确的labels, 没有分配GT的点, 分配的labels是-1(即负样本)
    #         # Interface for possible user-defined properties
    #         self._extra_properties = {}

    return num_gt, assigned_gt_inds, max_overlaps, assigned_labels


In [4]:
import  numpy as np
import itertools as it
class GeneratDefaultGridCells:
    def __init__(self):
        self.center_priors = []
        anchor_size = np.array([8,16,32])
        for i, feature_size in enumerate([40,20,10]):
            w, h = anchor_size[i], anchor_size[i]
            for i, j in it.product(range(feature_size), repeat=2):
                cx, cy = (j + 0.5) * w, (i + 0.5) * w
                self.center_priors.append([cy, cx, h, w])

        def to_ltrb(cy, cx, h, w):
            return cy - h / 2, cx - w / 2, cy + h / 2, cx + w / 2

        self.center_priors = np.array(self.center_priors, dtype='float32')
        self.center_priors_ltrb = np.array(tuple(to_ltrb(*i) for i in self.center_priors), dtype='float32')

center_priors = GeneratDefaultGridCells().center_priors
center_priors_ltrb = GeneratDefaultGridCells().center_priors_ltrb

In [5]:
# bboxes, num_level_bboxes, gt_bboxes, gt_bboxes_ignore=None, gt_labels=None
num_level_bboxes = [1600, 400, 100]
bboxes = torch.FloatTensor(center_priors_ltrb)
gt_bboxes = torch.FloatTensor([[117.396454 , 29.538464, 138.22485,  100.92308]])
gt_labels = torch.FloatTensor([37])

In [37]:
num_gt, assigned_gt_inds, max_overlaps, assigned_labels = assign(bboxes,num_level_bboxes,gt_bboxes,None,gt_labels)
print("!")

KeyboardInterrupt: 